In [135]:
import os
import requests
import pandas as pd
import numpy as np
import json
import time
import random
from lxml import etree
import re

#### 电影《春潮》评论的网址分析
url1 = 'https://movie.douban.com/subject/27186348/comments?start=0&limit=20&sort=new_score&status=P' 

url2 = 'https://movie.douban.com/subject/27186348/comments?start=20&limit=20&sort=new_score&status=P'

url3 = 'https://movie.douban.com/subject/27186348/comments?start=40&limit=20&sort=new_score&status=P'

url4 = 'https://movie.douban.com/subject/27186348/comments?start=60&limit=20&sort=new_score&status=P'

可以看出评论信息通过动态加载，网址中唯一变动的是start，故可以通过调整start来更改网址，从而采集网址中的信息

且网页中的内容是封装在'https://movie.douban.com/subject/27186348/comments?start=60&limit=20&sort=new_score&status=P&comments_only=1' 中，故只需在网址中加入comments_only=1即可

In [136]:
# 构造待采集的网页
def get_url(num):
    '''
    num: 定义的采集网页的页数
    '''
    urls = []
    base_url = 'https://movie.douban.com/subject/27186348/comments?start={}&limit=40&sort=new_score&status=P'
    for i in range(0, 20*num, 20):
        url = base_url.format(i)
        urls.append(url)
    
    return urls

# 采集前11页
urls = get_url(11) 

In [137]:
# 伪装请求头，还可以加入其它内容，比如cookies等
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36'
}

In [138]:
# 解析网页
def parse_movie_comments(url, headers):
    html = requests.get(url, headers=headers)
    bs = etree.HTML(html_get.text)
    # 评论的用户名
    user_name = bs.xpath('//div[@id="comments"]//span[@class="comment-info"]//a/text()')
    # 评分
    rate_str = bs.xpath('//div[@id="comments"]//span[@class="comment-info"]//span/@class')
    rate_star = []
    for i in range(0, len(rate_str), 2):
        rate_tep = re.compile(r'\d+').findall(rate_str[i])
        star = int(rate_tep[0]) / 10
        rate_star.append(star)
    # 推荐等级和评论时间
    reviews = bs.xpath('//div[@id="comments"]//span[@class="comment-info"]//span/@title')
    # 推荐等级
    reviews_level = []
    for i in range(0, len(reviews), 2):
        level = reviews[i]
        reviews_level.append(level)
    # 评论时间
    reviews_time = []
    for i in range(1, len(reviews), 2):
        time_tep = reviews[i]
        reviews_time.append(time_tep)
    # 评论内容
    comments = bs.xpath('//div[@id="comments"]//span[@class="short"]/text()')

    concat_info = pd.DataFrame({
        '用户': user_name,
        '评分': rate_star,
        '推荐等级': reviews_level,
        '评论时间': reviews_time,
        '评论内容': comments
    })
    
    return concat_info 

In [139]:
com_info = parse_movie_comments(urls[0], headers)
com_info.head()

,用户,评分,推荐等级,评论时间,评论内容
0,影志,4.0,推荐,2019-06-18 18:42:27,最有趣的就是小女儿的那些东北台词了：姥姥你这么说你丈夫合适吗、周叔叔你这马屁拍的都可以做教科...
1,二月鸟语,3.0,还行,2019-07-23 01:19:28,3.5/10，看起来是给郝蕾定做的，实则金燕玲的形象才是塑造和表演最好的。和那些只管灌输情绪...
2,木卫二,3.0,还行,2019-06-23 21:25:16,看得好累😫。导演大致借鉴了《一一》金燕玲的几场掏心戏（因为沟通已经不可能，哪怕结尾安排郝蕾来...
3,桃桃林林,4.0,推荐,2019-06-18 19:13:12,7分。有尬的地方，也有很妙的地方，但整体还是喜欢的。讲母女关系，却又远不仅是讲母女关系，这就...
4,silviaw,5.0,力荐,2019-06-18 17:48:13,纪念第一次在电影院看郝蕾。


In [140]:
# 批量采集评论信息
comments_result = pd.DataFrame()
count = 1
for url in urls:
    df = parse_movie_comments(url, headers)
    comments_result = pd.concat([comments_result, df])
    time.sleep(random.random() + 2)
    print('目前在采集第{}页'.format(count))
    count += 1

comments_result.head()

目前在采集第1页
目前在采集第2页
目前在采集第3页
目前在采集第4页
目前在采集第5页
目前在采集第6页
目前在采集第7页
目前在采集第8页
目前在采集第9页
目前在采集第10页
目前在采集第11页


,用户,评分,推荐等级,评论时间,评论内容
0,影志,4.0,推荐,2019-06-18 18:42:27,最有趣的就是小女儿的那些东北台词了：姥姥你这么说你丈夫合适吗、周叔叔你这马屁拍的都可以做教科...
1,二月鸟语,3.0,还行,2019-07-23 01:19:28,3.5/10，看起来是给郝蕾定做的，实则金燕玲的形象才是塑造和表演最好的。和那些只管灌输情绪...
2,木卫二,3.0,还行,2019-06-23 21:25:16,看得好累😫。导演大致借鉴了《一一》金燕玲的几场掏心戏（因为沟通已经不可能，哪怕结尾安排郝蕾来...
3,桃桃林林,4.0,推荐,2019-06-18 19:13:12,7分。有尬的地方，也有很妙的地方，但整体还是喜欢的。讲母女关系，却又远不仅是讲母女关系，这就...
4,silviaw,5.0,力荐,2019-06-18 17:48:13,纪念第一次在电影院看郝蕾。


In [143]:
# 输出采集到的影评信息
comments_result.to_csv('《春潮》影评信息.csv', index=0, encoding='utf-8')